In [1]:
import pandas as pd
import numpy as np
from datetime import datetime, timedelta
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_absolute_error, r2_score
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import DataLoader, TensorDataset

# Create a date range for the dataset
start_date = datetime(2020, 1, 1)
end_date = datetime(2021, 12, 31)
date_range = [start_date + timedelta(days=i) for i in range((end_date - start_date).days + 1)]

# Create an empty DataFrame to store the mock-up dataset
data = {
    "Month": [],  # Changed from "Date" to "Month"
    "Seasonality": [],
    "MarketTrends": [],
    "Temperature": [],
    "CompetitorPrices": [],
    "Demand": []
}

# Generate data for each date in the date range
num_samples = 25000  # Number of rows in the dataset
for _ in range(num_samples):
    date = np.random.choice(date_range)  # Randomly select a date
    seasonality = np.sin(2 * np.pi * date.month / 12)  # Simulated seasonal pattern
    market_trends = seasonality + np.random.uniform(-0.1, 0.1)  # Simulated market trends with noise
    temperature = np.random.uniform(10, 30)  # Simulated temperature data
    competitor_prices = 100 + seasonality * 50 + np.random.normal(0, 10)  # Simulated competitor prices
    
    # Create a simple linear relationship with demand
    demand = 200 - (2 * seasonality) + (3 * market_trends) + (4 * temperature) + (1.5 * competitor_prices) + np.random.normal(0, 20)
    
    # Extract the month from the date
    month = date.month
    
    data["Month"].append(month)
    data["Seasonality"].append(seasonality)
    data["MarketTrends"].append(market_trends)
    data["Temperature"].append(temperature)
    data["CompetitorPrices"].append(competitor_prices)
    data["Demand"].append(demand)

# Create the DataFrame
mockup_dataset = pd.DataFrame(data)

# Save the dataset to a CSV file
mockup_dataset.to_csv("demand_data_25000.csv", index=False)

# Load the demand data from the CSV file
data = pd.read_csv("demand_data_25000.csv")
data

,Month,Seasonality,MarketTrends,Temperature,CompetitorPrices,Demand
0,12,-2.449294e-16,0.089923,12.684690,96.105682,395.720702
1,6,1.224647e-16,-0.039619,18.045963,99.120044,406.852048
2,5,5.000000e-01,0.593797,25.505666,123.586310,457.156021
3,8,-8.660254e-01,-0.884742,18.011858,57.078076,359.529674
4,2,8.660254e-01,0.840033,11.709165,143.116653,437.992781
...,...,...,...,...,...,...
24995,5,5.000000e-01,0.578525,24.714731,120.530876,476.426605
24996,4,8.660254e-01,0.841502,16.099683,147.615646,483.445444
24997,1,5.000000e-01,0.459348,19.226013,127.704005,427.051179
24998,5,5.000000e-01,0.458301,14.210408,121.304115,390.205671


In [2]:
from sklearn.preprocessing import StandardScaler
# Define the target variable (Demand) and features, including 'Month'
target = "Demand"
features = ["Month", "Seasonality", "MarketTrends", "Temperature", "CompetitorPrices"]

# Split the data into features (X) and target variable (y)
X = data[features].values
y = data[target].values

# Split the data into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Standardize the features (recommended for neural networks)
scaler = StandardScaler()
X_train = scaler.fit_transform(X_train)
X_test = scaler.transform(X_test)

# Convert data to PyTorch tensors
X_train_tensor = torch.tensor(X_train, dtype=torch.float32)
y_train_tensor = torch.tensor(y_train, dtype=torch.float32)
X_test_tensor = torch.tensor(X_test, dtype=torch.float32)
y_test_tensor = torch.tensor(y_test, dtype=torch.float32)

# Define a simple feedforward neural network model
class FeedForwardNN(nn.Module):
    def __init__(self, input_size, hidden_size, output_size):
        super(FeedForwardNN, self).__init__()
        self.fc1 = nn.Linear(input_size, hidden_size)
        self.relu = nn.ReLU()
        self.fc2 = nn.Linear(hidden_size, output_size)

    def forward(self, x):
        x = self.fc1(x)
        x = self.relu(x)
        x = self.fc2(x)
        return x

# Initialize the neural network model
input_size = len(features)
hidden_size = 64
output_size = 1
model = FeedForwardNN(input_size, hidden_size, output_size)

# Define the loss function and optimizer
criterion = nn.MSELoss()  # Mean Squared Error loss for regression
optimizer = optim.Adam(model.parameters(), lr=0.001)

# Create DataLoader for batch training (optional but recommended for large datasets)
batch_size = 32
train_dataset = TensorDataset(X_train_tensor, y_train_tensor)
train_loader = DataLoader(train_dataset, batch_size=batch_size, shuffle=True)

# Training loop
num_epochs = 100
for epoch in range(num_epochs):
    model.train()
    for inputs, targets in train_loader:
        optimizer.zero_grad()
        outputs = model(inputs)
        loss = criterion(outputs, targets.view(-1, 1))  # Reshape targets for MSE loss
        loss.backward()
        optimizer.step()

# Evaluation
model.eval()
with torch.no_grad():
    y_pred_tensor = model(X_test_tensor)
    y_pred = y_pred_tensor.numpy()

# Convert predictions and ground truth to numpy arrays
y_test_np = y_test_tensor.numpy()

# Evaluate the model
mae = mean_absolute_error(y_test_np, y_pred)
r2 = r2_score(y_test_np, y_pred)

print(f"Mean Absolute Error (MAE): {mae:.2f}")
print(f"R-squared (R²) Score: {r2:.4f}")

Mean Absolute Error (MAE): 16.04
R-squared (R²) Score: 0.9013


In [3]:
import random
# Generate demand forecasts for each month using the trained model
forecasted_demand = []
for month in range(1, 13):
    input_data = torch.tensor([[month] + [random.uniform(-1, 1), random.uniform(-0.1, 0.1), random.uniform(10, 30), random.uniform(50, 150)]], dtype=torch.float32)
    forecast = model(input_data).item()
    forecasted_demand.append(forecast)

# Create a DataFrame for the demand forecasts
months = list(range(1, 13))
demand_df = pd.DataFrame({"Month": months, "Forecasted Demand (TEU)": forecasted_demand})
print(demand_df)


    Month  Forecasted Demand (TEU)
0       1              6865.305664
1       2              7690.022461
2       3              8708.687500
3       4              7204.770508
4       5              8512.125000
5       6              6452.749023
6       7              7931.142578
7       8              8816.700195
8       9              7793.948242
9      10              5527.745117
10     11              9002.771484
11     12              5060.149902


In [4]:
import pandas as pd
import random

# Number of containers
num_containers = 2000

# Create a list of container IDs (e.g., C001, C002, ...)
container_ids = ["C{:03d}".format(i) for i in range(1, num_containers + 1)]

# Generate random capacities for the containers (e.g., between 20 and 100 TEU)
capacities = [random.randint(20, 100) for _ in range(num_containers)]

# Generate random shipping status for the containers (0 for not shipping, 1 for shipping)
shipping_status = [random.choice([0, 1]) for _ in range(num_containers)]

# Create a DataFrame for the supply dataset
supply_data = {
    "ContainerID": container_ids,
    "Capacity (TEU)": capacities,
    "Shipping Status": shipping_status
}

supply_df = pd.DataFrame(supply_data)
supply_df

,ContainerID,Capacity (TEU),Shipping Status
0,C001,58,0
1,C002,81,1
2,C003,86,1
3,C004,57,0
4,C005,98,0
...,...,...,...
1995,C1996,29,1
1996,C1997,67,0
1997,C1998,42,1
1998,C1999,58,1


In [5]:
# Create a dictionary to map month numbers to their names
month_names = {
    1: "Jan",
    2: "Feb",
    3: "Mar",
    4: "Apr",
    5: "May",
    6: "Jun",
    7: "Jul",
    8: "Aug",
    9: "Sep",
    10: "Oct",
    11: "Nov",
    12: "Dec"
}

# Create a list to store allocation DataFrames for each day of the year
allocation_dfs = []

# Create a dictionary to track the shipping status of containers
container_shipping_status = {}

# Iterate through each month
for _, demand_row in demand_df.iterrows():
    month = demand_row["Month"]
    demand = demand_row["Forecasted Demand (TEU)"]

    # Create a copy of the supply dataset for this month
    allocation_df = supply_df.copy()
    allocation_df["Allocated (TEU)"] = 0  # Initialize the allocated column

    # Filter containers to include only those with Shipping Status equal to 1
    eligible_containers = allocation_df[allocation_df["Shipping Status"] == 1]
    
    # Shuffle the eligible containers randomly for each month
    eligible_containers = eligible_containers.sample(frac=1)
    
    # Calculate daily allocation
    daily_allocation = demand / 31  # Divide monthly demand by the number of days in the month
    
    allocated_container_ids = []
    # Filter containers with shipping status equal to 0 and add their IDs to allocated_container_ids
    for idx, status in enumerate(shipping_status):
        if status == 0:
            allocated_container_ids.append(container_ids[idx])
    
    # Allocate containers to meet demand for each day of the current month
    for day in range(1, 32):
        
        # Reset daily demand to the calculated daily allocation
        daily_demand = daily_allocation
        
        # Create a list to store allocated container IDs for this day
        # Create a list to store allocated container IDs with shipping status equal to 0
        # Filter out already allocated containers from eligible containers
        eligible_containers = eligible_containers[~eligible_containers["ContainerID"].isin(allocated_container_ids)]
        
        allocated_container_ids = []
        
        for _, container_row in eligible_containers.iterrows():
            container_id = container_row["ContainerID"]
            
            capacity = container_row["Capacity (TEU)"]

            if daily_demand >= capacity:
                allocation_df.at[_, "Allocated (TEU)"] = capacity
                daily_demand -= capacity
                allocated_container_ids.append(container_id)  # Add allocated container ID to the list
                
                # Set the shipping status of the container to 0
                container_shipping_status[container_id] = 20  # Set to 20 days (unavailable for 20 days)
            else:
                allocation_df.at[_, "Allocated (TEU)"] = daily_demand
                daily_demand = 0
                allocated_container_ids.append(container_id)  # Add allocated container ID to the list
                
                # Set the shipping status of the container to 0
                container_shipping_status[container_id] = 20  # Set to 20 days (unavailable for 20 days)

            if daily_demand == 0:
                break
        
        # Decrement the shipping status of containers that were unavailable
        for container_id in container_shipping_status:
            if container_shipping_status[container_id] > 0:
                container_shipping_status[container_id] -= 1
        
        # Print detailed information of allocated containers for this day
        print(f"Detailed Information of Allocated Containers for {month}, Day {day}:")
        allocated_containers_info = allocation_df[allocation_df["ContainerID"].isin(allocated_container_ids)]
        print(allocated_containers_info)
        print("\n")

    # Append the allocation DataFrame for this month to the list
    allocation_dfs.append(allocation_df.copy())

Detailed Information of Allocated Containers for 1.0, Day 1:
     ContainerID  Capacity (TEU)  Shipping Status  Allocated (TEU)
122         C123              96                1        88.461473
776         C777              61                1        61.000000
1628       C1629              72                1        72.000000


Detailed Information of Allocated Containers for 1.0, Day 2:
    ContainerID  Capacity (TEU)  Shipping Status  Allocated (TEU)
198        C199              88                1        88.000000
243        C244              46                1        46.000000
338        C339              74                1         5.461473
647        C648              82                1        82.000000


Detailed Information of Allocated Containers for 1.0, Day 3:
     ContainerID  Capacity (TEU)  Shipping Status  Allocated (TEU)
437         C438              47                1        47.000000
588         C589              84                1        84.000000
1162       C11

Detailed Information of Allocated Containers for 1.0, Day 26:
     ContainerID  Capacity (TEU)  Shipping Status  Allocated (TEU)
302         C303              80                1         7.461473
413         C414              70                1        70.000000
1051       C1052              49                1        49.000000
1160       C1161              57                1        57.000000
1267       C1268              38                1        38.000000


Detailed Information of Allocated Containers for 1.0, Day 27:
     ContainerID  Capacity (TEU)  Shipping Status  Allocated (TEU)
224         C225              73                1        73.000000
1069       C1070             100                1        18.461473
1741       C1742              78                1        78.000000
1942       C1943              52                1        52.000000


Detailed Information of Allocated Containers for 1.0, Day 28:
     ContainerID  Capacity (TEU)  Shipping Status  Allocated (TEU)
150   

Detailed Information of Allocated Containers for 2.0, Day 20:
     ContainerID  Capacity (TEU)  Shipping Status  Allocated (TEU)
29          C030              53                1        53.000000
932         C933              83                1        83.000000
1593       C1594              35                1        35.000000
1663       C1664              36                1        36.000000
1775       C1776              65                1        41.065241


Detailed Information of Allocated Containers for 2.0, Day 21:
     ContainerID  Capacity (TEU)  Shipping Status  Allocated (TEU)
478         C479              48                1        48.000000
496         C497              53                1        53.000000
520         C521              71                1        34.065241
1449       C1450              86                1        86.000000
1568       C1569              27                1        27.000000


Detailed Information of Allocated Containers for 2.0, Day 22:
     C

Detailed Information of Allocated Containers for 3.0, Day 15:
     ContainerID  Capacity (TEU)  Shipping Status  Allocated (TEU)
451         C452              36                1        36.000000
649         C650              94                1        94.000000
921         C922              42                1        42.000000
1066       C1067              76                1         3.925403
1552       C1553              76                1        76.000000
1995       C1996              29                1        29.000000


Detailed Information of Allocated Containers for 3.0, Day 16:
     ContainerID  Capacity (TEU)  Shipping Status  Allocated (TEU)
1048       C1049              75                1        14.925403
1149       C1150              82                1        82.000000
1284       C1285              73                1        73.000000
1321       C1322              69                1        69.000000
1445       C1446              42                1        42.000000


D

Detailed Information of Allocated Containers for 4.0, Day 9:
     ContainerID  Capacity (TEU)  Shipping Status  Allocated (TEU)
301         C302              60                1        60.000000
820         C821              97                1        79.411952
1487       C1488              36                1        36.000000
1954       C1955              57                1        57.000000


Detailed Information of Allocated Containers for 4.0, Day 10:
    ContainerID  Capacity (TEU)  Shipping Status  Allocated (TEU)
381        C382              66                1        66.000000
613        C614              46                1        46.000000
728        C729              66                1        66.000000
969        C970              65                1        54.411952


Detailed Information of Allocated Containers for 4.0, Day 11:
     ContainerID  Capacity (TEU)  Shipping Status  Allocated (TEU)
98          C099              51                1        51.000000
241         

...

Detailed Information of Allocated Containers for 9.0, Day 14:
     ContainerID  Capacity (TEU)  Shipping Status  Allocated (TEU)
474         C475              49                1        49.000000
847         C848              96                1        96.000000
1007       C1008              89                1        89.000000
1617       C1618              86                1        17.417685


Detailed Information of Allocated Containers for 9.0, Day 15:
     ContainerID  Capacity (TEU)  Shipping Status  Allocated (TEU)
192         C193              63                1        17.417685
613         C614              46                1        46.000000
1499       C1500              99                1        99.000000
1867       C1868              89                1        89.000000


Detailed Information of Allocated Containers for 9.0, Day 16:
     ContainerID  Capacity (TEU)  Shipping Status  Allocated (TEU)
290         C291              60                1        46.417685
616   

Detailed Information of Allocated Containers for 10.0, Day 5:
     ContainerID  Capacity (TEU)  Shipping Status  Allocated (TEU)
276         C277              48                1        48.000000
451         C452              36                1        36.000000
588         C589              84                1        84.000000
1670       C1671              75                1        10.314359


Detailed Information of Allocated Containers for 10.0, Day 6:
     ContainerID  Capacity (TEU)  Shipping Status  Allocated (TEU)
701         C702              49                1        49.000000
1315       C1316              88                1        88.000000
1795       C1796              83                1        16.314359
1799       C1800              25                1        25.000000


Detailed Information of Allocated Containers for 10.0, Day 7:
     ContainerID  Capacity (TEU)  Shipping Status  Allocated (TEU)
283         C284              59                1        59.000000
585   

Detailed Information of Allocated Containers for 11.0, Day 1:
     ContainerID  Capacity (TEU)  Shipping Status  Allocated (TEU)
2           C003              86                1         6.411983
535         C536              39                1        39.000000
540         C541              28                1        28.000000
922         C923              49                1        49.000000
950         C951              52                1        52.000000
1629       C1630              74                1        74.000000
1660       C1661              42                1        42.000000


Detailed Information of Allocated Containers for 11.0, Day 2:
     ContainerID  Capacity (TEU)  Shipping Status  Allocated (TEU)
99          C100              26                1        26.000000
305         C306              35                1         7.411983
578         C579              92                1        92.000000
622         C623              27                1        27.000000
642

Detailed Information of Allocated Containers for 11.0, Day 26:
     ContainerID  Capacity (TEU)  Shipping Status  Allocated (TEU)
29          C030              53                1        53.000000
597         C598              24                1        14.411983
1071       C1072              45                1        45.000000
1117       C1118              96                1        96.000000
1306       C1307              82                1        82.000000


Detailed Information of Allocated Containers for 11.0, Day 27:
     ContainerID  Capacity (TEU)  Shipping Status  Allocated (TEU)
218         C219              98                1        98.000000
373         C374              99                1        99.000000
379         C380              59                1        26.411983
1923       C1924              67                1        67.000000


Detailed Information of Allocated Containers for 11.0, Day 28:
     ContainerID  Capacity (TEU)  Shipping Status  Allocated (TEU)
98 

Detailed Information of Allocated Containers for 12.0, Day 21:
     ContainerID  Capacity (TEU)  Shipping Status  Allocated (TEU)
1163       C1164              99                1        99.000000
1256       C1257              73                1        64.230642


Detailed Information of Allocated Containers for 12.0, Day 22:
     ContainerID  Capacity (TEU)  Shipping Status  Allocated (TEU)
699         C700              98                1        98.000000
956         C957              21                1        21.000000
1103       C1104              62                1        16.230642
1710       C1711              28                1        28.000000


Detailed Information of Allocated Containers for 12.0, Day 23:
    ContainerID  Capacity (TEU)  Shipping Status  Allocated (TEU)
36         C037              70                1        30.230642
122        C123              96                1        96.000000
418        C419              37                1        37.000000


Detai

In [ ]:
import pandas as pd
import numpy as np
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import TensorDataset, DataLoader
from sklearn.preprocessing import StandardScaler

# ... (Code for creating demand and supply DataFrames and training the model) ...

# Create a dictionary to map month names to their numbers
month_numbers = {
    1: "Jan",
    2: "Feb",
    3: "Mar",
    4: "Apr",
    5: "May",
    6: "Jun",
    7: "Jul",
    8: "Aug",
    9: "Sep",
    10: "Oct",
    11: "Nov",
    12: "Dec"
}

# Define a function to perform allocation and forecasting for a specific month
def allocate_and_forecast(month, seasonality, market_trends, temperature, competitor_prices):
    # Get the month number from its name
    month_num = int(month)
    
    # Check if the month number is valid
    if month_num < 1 or month_num > 12:
        print("Invalid month input. Please enter a number between 1 and 12.")
        return
    
    # Convert input features to a PyTorch tensor
    input_data = torch.tensor([[month_num, seasonality, market_trends, temperature, competitor_prices]], dtype=torch.float32)
    
    # Use the trained model to forecast demand for the specified month
    forecasted_demand = model(input_data).item()
    print(f"Forecasted Demand for {month}: {forecasted_demand:.2f} TEU")
    
    # Create a copy of the supply dataset for this month
    allocation_df = supply_df.copy()
    allocation_df["Allocated (TEU)"] = 0  # Initialize the allocated column

    # Filter containers to include only those with Shipping Status equal to 1
    eligible_containers = allocation_df[allocation_df["Shipping Status"] == 1]
    
    # Shuffle the eligible containers randomly for each month
    eligible_containers = eligible_containers.sample(frac=1)
    
    # Calculate daily allocation
    daily_allocation = forecasted_demand / 31  # Divide monthly demand by the number of days in the month
    
    allocated_container_ids = []
    # Filter containers with shipping status equal to 0 and add their IDs to allocated_container_ids
    for idx, status in enumerate(shipping_status):
        if status == 0:
            allocated_container_ids.append(container_ids[idx])
    
    # Allocate containers to meet demand for each day of the current month
    for day in range(1, 32):
        
        # Reset daily demand to the calculated daily allocation
        daily_demand = daily_allocation
        
        # Filter out already allocated containers from eligible containers
        eligible_containers = eligible_containers[~eligible_containers["ContainerID"].isin(allocated_container_ids)]
        
        # Create a list to store allocated container IDs for this day
        allocated_container_ids = []
        
        for _, container_row in eligible_containers.iterrows():
            container_id = container_row["ContainerID"]
            
            capacity = container_row["Capacity (TEU)"]

            if daily_demand >= capacity:
                allocation_df.at[_, "Allocated (TEU)"] = capacity
                daily_demand -= capacity
                allocated_container_ids.append(container_id)  # Add allocated container ID to the list
                
                # Set the shipping status of the container to 0
                container_shipping_status[container_id] = 20  # Set to 20 days (unavailable for 20 days)
            else:
                allocation_df.at[_, "Allocated (TEU)"] = daily_demand
                daily_demand = 0
                allocated_container_ids.append(container_id)  # Add allocated container ID to the list
                
                # Set the shipping status of the container to 0
                container_shipping_status[container_id] = 20  # Set to 20 days (unavailable for 20 days)

            if daily_demand == 0:
                break
        
        # Decrement the shipping status of containers that were unavailable
        for container_id in container_shipping_status:
            if container_shipping_status[container_id] > 0:
                container_shipping_status[container_id] -= 1
        
        # Print detailed information of allocated containers for this day
        print(f"Detailed Information of Allocated Containers for {month}, Day {day}:")
        allocated_containers_info = allocation_df[allocation_df["ContainerID"].isin(allocated_container_ids)]
        print(allocated_containers_info)
        print("\n")

# Interactive input
month = int(input("Enter the month (e.g., '1-12'): "))
seasonality = float(input("Enter the seasonality factor: "))
market_trends = float(input("Enter the market trends factor: "))
temperature = float(input("Enter the temperature: "))
competitor_prices = float(input("Enter the competitor prices: "))

# Call the allocate_and_forecast function with user input
allocate_and_forecast(month, seasonality, market_trends, temperature, competitor_prices)